In [8]:
print("LA STRATÉGIE GAGNANTE : LE FINE-TUNING MAINTENANT , UTILISE QUE CE QUI EST ECRIT EN DESSOUS SI TU A CE MODèLE A ENTRAINER")

LA STRATÉGIE GAGNANTE : LE FINE-TUNING MAINTENANT , UTILISE QUE CE QUI EST ECRIT EN DESSOUS SI TU A CE MODèLE A ENTRAINER


In [10]:
import os
import json
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from torch.cuda.amp import GradScaler, autocast

# --- CONFIGURATION ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")
BATCH_SIZE = 128 # On profite de la 4070
DEVICE = torch.device("cuda")

print(f"🚀 RECONSTRUCTION DU MODELE LEGENDAIRE (91%) SUR : {DEVICE}")

# --- DATASET ---
# On utilise la transformation STANDARD ImageNet (celle utilisée pour créer les .npy implicitement)
trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), # Le zoom standard ResNet
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)
df['path'] = df.apply(lambda x: os.path.join(IMG_DIR, f"image_{x['imageid']}_product_{x['productid']}.jpg"), axis=1)

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
NUM_CLASSES = len(le.classes_)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

class LegendDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try: img = Image.open(row['path']).convert("RGB")
        except: img = Image.new('RGB', (224, 224), (0, 0, 0))
        if self.transform: img = self.transform(img)
        return img, torch.tensor(row['label_encoded'], dtype=torch.long)

train_loader = DataLoader(LegendDataset(train_df, trans), batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(LegendDataset(val_df, trans), batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# --- ARCHITECTURE EXACTE DU LOG (Rang DL 1) ---
# L:[2048, 1024, 512] | Opt:adam | Act:gelu | Drop:0.2

class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        # Reconstruction exacte de ta classe ModularMLP gagnante
        self.net = nn.Sequential(
            # Couche 1 : 2048 -> 2048 (Input ResNet -> Hidden 1)
            nn.Linear(input_dim, 2048),
            nn.BatchNorm1d(2048),
            nn.GELU(), # Activation GELU (C'était le secret !)
            nn.Dropout(0.2),
            
            # Couche 2 : 2048 -> 1024
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.2),
            
            # Couche 3 : 1024 -> 512
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(0.2),
            
            # Sortie : 512 -> 27
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        return self.net(x)

# Assemblage avec ResNet50
resnet = models.resnet50(weights="IMAGENET1K_V1")

# ON GÈLE TOUT LE RESNET (Comme si on utilisait des .npy fixes)
for param in resnet.parameters():
    param.requires_grad = False

# On remplace la tête
resnet.fc = LegendMLP(2048, NUM_CLASSES)
model = resnet.to(DEVICE)

print("🧠 Architecture 'Legend' (GELU + Config Exacte) reconstruite.")

# --- ENTRAINEMENT (REPLIQUE DU LOG) ---
criterion = nn.CrossEntropyLoss()
# Optimiseur ADAM (Pas AdamW, comme dans ton log)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001) 
scaler = GradScaler()

print("🔥 Démarrage (Objectif : Retrouver les 91%)...")
EPOCHS = 20 # Comme dans ton log
best_f1 = 0.0

for epoch in range(EPOCHS):
    model.train()
    t0 = time.time()
    
    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(DEVICE), lbls.to(DEVICE)
        
        optimizer.zero_grad()
        with autocast():
            out = model(imgs)
            loss = criterion(out, lbls)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        if i % 50 == 0: print(f"   ⏳ Ep {epoch+1} | Batch {i} | Loss: {loss.item():.4f}", end="\r")

    # Validation
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs = imgs.to(DEVICE)
            with autocast(): out = model(imgs)
            _, p = torch.max(out, 1)
            preds.extend(p.cpu().numpy())
            targets.extend(lbls.cpu().numpy())
    
    val_f1 = f1_score(targets, preds, average='weighted')
    duree = time.time() - t0
    
    print(f"\n✅ FIN EP {epoch+1} | Time: {duree:.0f}s | F1: {val_f1:.4f}")
    
    # Sauvegarde systématique si record
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "livrable_model_resnet_reborn.pth"))
        
        # Metadata pour le Voting
        meta_data = {
            "model_name": "ResNet50 Legend 91%", 
            "class_mapping": {int(i): str(c) for i, c in enumerate(le.classes_)}
        }
        with open(os.path.join(OUTPUT_DIR, "livrable_resnet_reborn_metadata.json"), 'w') as f:
            json.dump(meta_data, f, indent=4)
        print("   💾 Record Sauvegardé.")

print(f"🏆 Fini. Score Final : {best_f1:.4f}")

🚀 RECONSTRUCTION DU MODELE LEGENDAIRE (91%) SUR : cuda
🧠 Architecture 'Legend' (GELU + Config Exacte) reconstruite.
🔥 Démarrage (Objectif : Retrouver les 91%)...


C:\Users\amisf\AppData\Local\Temp\ipykernel_44200\3707796505.py:112: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\amisf\AppData\Local\Temp\ipykernel_44200\3707796505.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ⏳ Ep 1 | Batch 500 | Loss: 1.7137

C:\Users\amisf\AppData\Local\Temp\ipykernel_44200\3707796505.py:142: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)



✅ FIN EP 1 | Time: 415s | F1: 0.5418
   💾 Record Sauvegardé.


...

C:\Users\amisf\AppData\Local\Temp\ipykernel_44200\3707796505.py:142: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)



✅ FIN EP 7 | Time: 398s | F1: 0.6074


C:\Users\amisf\AppData\Local\Temp\ipykernel_44200\3707796505.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ⏳ Ep 8 | Batch 100 | Loss: 0.6717


KeyboardInterrupt



In [12]:
import os
import json
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

# --- CONFIGURATION ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"🚀 OPÉRATION PHOENIX SUR : {DEVICE}")

# 1. CHARGEMENT DU TRÉSOR (.NPY)
print("📥 Chargement des features pré-calculées (3.1 Go)...")
try:
    # On charge les fichiers du 5 Janvier
    X_all = np.load(os.path.join(OUTPUT_DIR, 'train_features_resnet50_augmented.npy'))
    y_all_raw = np.load(os.path.join(OUTPUT_DIR, 'train_labels_augmented.npy'))
    print(f"✅ Features chargées : {X_all.shape}")
except Exception as e:
    raise FileNotFoundError(f"❌ Impossible de charger les .npy : {e}")

# 2. ENCODAGE & SPLIT
# On doit ré-encoder les labels proprement pour être cohérent avec le projet
le = LabelEncoder()
y_all = le.fit_transform(y_all_raw)
NUM_CLASSES = len(le.classes_)

print("✂️ Création du Train/Val...")
X_train, X_val, y_train, y_val = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42, stratify=y_all
)

# Conversion en Tenseurs PyTorch (Directement en VRAM pour vitesse lumière)
train_ds = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32).to(DEVICE), 
    torch.tensor(y_train, dtype=torch.long).to(DEVICE)
)
val_ds = TensorDataset(
    torch.tensor(X_val, dtype=torch.float32).to(DEVICE), 
    torch.tensor(y_val, dtype=torch.long).to(DEVICE)
)

# Gros batch size car ce ne sont pas des images, juste des vecteurs
train_loader = DataLoader(train_ds, batch_size=4096, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=4096, shuffle=False)

# 3. L'ARCHITECTURE "LEGEND" (Celle de ton log à 91%)
# Config : [2048, 1024, 512], Adam, GELU, Drop 0.2
class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.BatchNorm1d(2048), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(2048, 1024), nn.BatchNorm1d(1024), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(1024, 512),  nn.BatchNorm1d(512),  nn.GELU(), nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

mlp = LegendMLP(2048, NUM_CLASSES).to(DEVICE)
print("🧠 Cerveau MLP initialisé.")

# 4. ENTRAINEMENT ÉCLAIR (Sur les features, pas les images)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001) # Adam standard comme avant

print("🔥 Démarrage Entraînement PHOENIX (Ça va aller très vite)...")
EPOCHS = 20
best_f1 = 0.0

for epoch in range(EPOCHS):
    mlp.train()
    t0 = time.time()
    
    for bx, by in train_loader:
        optimizer.zero_grad()
        out = mlp(bx)
        loss = criterion(out, by)
        loss.backward()
        optimizer.step()

    # Validation
    mlp.eval()
    preds, targets = [], []
    with torch.no_grad():
        for bx, by in val_loader:
            out = mlp(bx)
            _, p = torch.max(out, 1)
            preds.extend(p.cpu().numpy())
            targets.extend(by.cpu().numpy())
            
    val_f1 = f1_score(targets, preds, average='weighted')
    duree = time.time() - t0
    print(f"✅ EP {epoch+1}/{EPOCHS} | Time: {duree:.1f}s | F1: {val_f1:.4f}")
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        # On sauvegarde juste les poids du MLP pour l'instant
        torch.save(mlp.state_dict(), os.path.join(OUTPUT_DIR, "temp_phoenix_mlp.pth"))

print(f"🏆 Score PHOENIX atteint : {best_f1:.4f}")

# 5. ASSEMBLAGE FINAL (FUSION CORPS + TÊTE) & SAUVEGARDE POUR VOTING
print("\n🏗️ Assemblage du modèle final pour le Voting...")

# On prend un ResNet vierge
full_model = models.resnet50(weights="IMAGENET1K_V1")
# On gèle le corps (puisqu'on a entraîné le MLP sur des features figées)
for param in full_model.parameters():
    param.requires_grad = False

# On lui greffe notre MLP entraîné
full_model.fc = LegendMLP(2048, NUM_CLASSES)
full_model.fc.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "temp_phoenix_mlp.pth")))

# On sauvegarde le TOUT (Corps + Tête) sous le nom que le Voting attend
final_path = os.path.join(OUTPUT_DIR, "livrable_model_resnet_reborn.pth")
torch.save(full_model.state_dict(), final_path)

# Metadata
meta_data = {
    "model_name": "ResNet50 Phoenix (From NPY)", 
    "class_mapping": {int(i): str(c) for i, c in enumerate(le.classes_)}
}
with open(os.path.join(OUTPUT_DIR, "livrable_resnet_reborn_metadata.json"), 'w') as f:
    json.dump(meta_data, f, indent=4)

print(f"💾 SAUVEGARDE TERMINÉE : {final_path}")
print("🚀 Tu peux lancer le Voting, le champion est de retour !")

🚀 OPÉRATION PHOENIX SUR : cuda
📥 Chargement des features pré-calculées (3.1 Go)...
✅ Features chargées : (405000, 2048)
✂️ Création du Train/Val...
🧠 Cerveau MLP initialisé.
🔥 Démarrage Entraînement PHOENIX (Ça va aller très vite)...
✅ EP 1/20 | Time: 3.6s | F1: 0.6585
✅ EP 2/20 | Time: 3.7s | F1: 0.7392
✅ EP 3/20 | Time: 3.7s | F1: 0.7783
✅ EP 4/20 | Time: 3.6s | F1: 0.8136
✅ EP 5/20 | Time: 3.7s | F1: 0.8340
✅ EP 6/20 | Time: 3.8s | F1: 0.8496
✅ EP 7/20 | Time: 3.7s | F1: 0.8693
✅ EP 8/20 | Time: 4.6s | F1: 0.8800
✅ EP 9/20 | Time: 3.7s | F1: 0.8829
✅ EP 10/20 | Time: 3.7s | F1: 0.8826
✅ EP 11/20 | Time: 4.1s | F1: 0.8936
✅ EP 12/20 | Time: 3.7s | F1: 0.8999
✅ EP 13/20 | Time: 3.8s | F1: 0.9026
✅ EP 14/20 | Time: 3.6s | F1: 0.9023
✅ EP 15/20 | Time: 3.7s | F1: 0.9064
✅ EP 16/20 | Time: 3.6s | F1: 0.9104
✅ EP 17/20 | Time: 3.7s | F1: 0.9085
✅ EP 18/20 | Time: 3.7s | F1: 0.9072
✅ EP 19/20 | Time: 3.8s | F1: 0.9125
✅ EP 20/20 | Time: 3.8s | F1: 0.9143
🏆 Score PHOENIX atteint : 0.9143

🏗